## Dieses Skript prepocesst die TUDA_battery_state.csv Dateien und speichert sie im pickle Format ab, somit muss prepocessing nur einmal durchgeführt werden

### Importieren der benötigten Bibliotheken

In [ ]:
### Importieren der verwendeten Bibilotheken
import os
import glob

import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

## Aufbau der Daten TUDA_battery_state.csv

aus dem Wiki aus dem moodle Kurs
+ wagon_ID: the wagon's ID	
+ provider: provider of the telematic module that is conducting the measurement
+ timestamp_measure_battery: timestamp of the measurement
+ battery_state: battery charge level of the device conducting the measurement
+ battery2_state: There is a provider that has a rechargable battery and a non-rechargable battery installed. Battery2 is then the non-rechargable battery charge state. For all other providers NaN is set.

### Einlesen der Daten
TUDA_battery_state.csv Dateien werden eingelesen.
Hinweis: können vollständig eingelesen werden, da die Dateien im Vergleich sehr klein sind.
Alle Dateien werden in ein pandas Dataframe eingelesen

In [ ]:
pathToData = os.path.join("..", "..", "data", "TUDA_battery_state",) #Pfad zu dem Ordner, in dem die csv Dateien gespeichert sind
filenames = glob.glob(pathToData + "/*.csv")

batteryDfList = [] # leere Liste für die DataFrames

for filename in filenames:
    df = pd.read_csv(filename)
    batteryDfList.append(df)

In [ ]:
# Alle Daten in einen einzelnen DataFrame
batteryDf = pd.concat(batteryDfList, axis=0, ignore_index=True) #axis 0 ist optional, weil default

# Löschen der korrupten Wochen aus dem Datensatz
#corrupted_weeks = np.add([18, 19, 20, 21, 32, 43, 45],-1)
#batteryDfList_clean = [v for i, v in enumerate(batteryDfList) if i not in corrupted_weeks]

#batteryDf_clean =  pd.concat(batteryDfList_clean, axis=0, ignore_index=True) #axis 0 ist optional, weil default

### Daten bereinigen, preprocessing

+ Prozentzeichen entfernen aus dem battery_state (manche haben ein Prozentzeichen, manche nicht)
+ umwandeln der Strings mit Prozentzeichen in floats

In [ ]:
def format_timestamp(timestamp_string):
    """
    Funktion die aus dem timestamp_string einen float in sekunden zurückgibt
    """
    string_list_leerzeichen = timestamp_string.split(' ')
    days = float(string_list_leerzeichen[0])
    days_in_sec = days * 24 * 60 * 60
    string_list_doppelpunkt = string_list_leerzeichen[-1].split(':')
    hours = float(string_list_doppelpunkt[0])
    hours_in_sec = hours * 60 * 60
    minutes = float(string_list_doppelpunkt[1])
    minutes_in_sec = minutes * 60
    sec_in_sec = float(string_list_doppelpunkt[2])
    total_time_in_sec = days_in_sec + hours_in_sec + minutes_in_sec + sec_in_sec
    return total_time_in_sec



In [ ]:
def preprocDF(df):
    # ersetzt die Strings mit Prozentzeichen, die in manchen Reihen auftreten durch floats
    # inplace=True gibt keine Ausgabe zurück sondern ersetzt existierenden Df
    # regex=True ersetzt explizit nur strings, sonst probleme mit den floats

    # Entfernen der Prozentzeichen aber weiterhin floats
    df['battery_state'].replace(r' %', '', inplace=True, regex=True)
    df['battery2_state'].replace(r' %', '', inplace=True, regex=True)

    # strings to floats
    df['battery_state'] = df['battery_state'].astype('float')
    df['battery2_state'] = df['battery2_state'].astype('float')

    ### umwandeln der wagonIDs in integers
    # die inneren Anführungszeichen '' entfernen
    df['wagon_ID'].replace('\'', '', inplace=True, regex=True)

    # strings to int
    df['wagon_ID'] = df['wagon_ID'].astype('int64')

    ### timestamp_measure_battery in geeignetes format
    # entfernen der Pluszeichen, da überflüssig
    df['timestamp_measure_battery'].replace('\+', '', inplace=True, regex=True)
    
    # entfernen von Zeilen mit NaT in timestamp
    df.drop(df.loc[df['timestamp_measure_battery']=='NaT'].index, inplace=True)
    
    # ändert die timestamps aus dem string in absolute Sekunden (auch auf 0-Punkt referenziert)
    df['timestamp_measure_battery'] = df['timestamp_measure_battery'].apply(format_timestamp)
    
    # entfernen von Zeilen mit negativen Zeitwerten
    # (diese wurden geplottet und als einzelne Ausreißer angesehen, ergeben für die weitere Bearbeitung keinen Sinn)
    df.drop(df.loc[df['timestamp_measure_battery']<0].index, inplace=True)

    return df

## Anwenden der Preprocess Funtion auf die Dfs

In [ ]:
batteryDf_preproc = preprocDF(batteryDf)

In [ ]:
#batteryDf_clean_preproc = preprocDF(batteryDf_clean)

### Einordnen der Werte in verschiedene Bins für eine bestimmte Zeitperiode


In [ ]:
# 'wagon_IDs' werden aus dem DataFrame ausgelesen
wagon_IDList_battery_state = np.sort(batteryDf_preproc["wagon_ID"].drop_duplicates().to_numpy())
wagon_IDList_battery_state

In [ ]:
len(wagon_IDList_battery_state)

In [ ]:
batteryDf_preproc.sort_values("wagon_ID")

### Erstellen von Tages-Bins

In [ ]:
# bins erstellen, in die die Daten eingeteilt und anschließend gemittelt werden sollen
bins_seconds = np.arange(0, 50*7*24*60*60, 1*24*60*60)
bins_seconds_labels = [i for i, bin in enumerate(bins_seconds)]
bins_seconds = bins_seconds.tolist()
bins_seconds_labels.pop() # ein Wert muss entfernt werden, um gleich viele Labels wie bins zu definieren!

In [ ]:
# Daten mittels der timestamps in bins einteilen (jeweils immer 1 Tag einen bin zuweisen)
batteryDf_preproc['Tag'] = pd.cut(batteryDf_preproc['timestamp_measure_battery'], bins = bins_seconds, labels = bins_seconds_labels)

### Erstellen von Wochen-Bins

In [ ]:
# bins erstellen, in die die Daten eingeteilt und anschließend gemittelt werden sollen
# bins werden erstellt, für jeden Woche zwischen 0 und 322, in Sekunden, da die Daten in Sekunden vorliegen
bins_seconds = np.arange(0, 50*7*24*60*60, 7*24*60*60)
# liste mit bin labels für jede Woche erstellen
bins_seconds_labels = [i+1 for i, bin in enumerate(bins_seconds)]
# bins werden aus numpy array in Liste umgewandelt
bins_seconds = bins_seconds.tolist()
# ein Wert muss entfernt werden, um gleich viele Labels wie bins zu definieren!
bins_seconds_labels.pop() 

In [ ]:
# Daten mittels der timestamps in bins einteilen (jeweils immer 1 Woche einen bin zuweisen)
batteryDf_preproc['Woche'] = pd.cut(batteryDf_preproc['timestamp_measure_battery'], bins = bins_seconds, labels = bins_seconds_labels)

In [ ]:
batteryDf_preproc.sort_values('Woche')

### Erstellen eines Df, indem die korrupeten Wochen entfernt wurden
+ nach Analysen ergab sich, dass Woche 22 und 23 auch korrupte Daten enthält, weswegen diese zusätzlich entfernt wurden

In [ ]:
batteryDf_clean_preproc = batteryDf_preproc.copy(deep=True)

corrupted_weeks = [18, 19, 20, 21, 22, 23, 32, 43, 45]
for week in corrupted_weeks:
    batteryDf_clean_preproc.drop(batteryDf_clean_preproc.loc[batteryDf_clean_preproc['Woche'] == week].index, inplace=True )

+ Entfernen der undefinierten Battierzustände

# Batteriezustände = 0 NICHT vernachlässigen, da diese nicht tatsächlich 0 sind --> laut Sprechstunde mit DBCargo --> folglich nur Werte kleiner 0 vernachlässigen
batteryDfbattery_state.drop(batteryDfbattery_state.loc[batteryDfbattery_state['battery_state']<0].index, inplace=True)
batteryDfbattery_state.dropna(inplace=True)

batteryDfbattery2_state.drop(batteryDfbattery2_state.loc[batteryDfbattery2_state['battery2_state']<0].index, inplace=True)
batteryDfbattery2_state.dropna(inplace=True)

In [ ]:
batteryDf_clean_preproc.drop(batteryDf_clean_preproc.loc[batteryDf_clean_preproc['battery_state'].isna()].index, inplace=True)

+ abspeichern des bereinigten DF in pickle-Datei

In [ ]:
# wird als gepickelte .csv Datei gespeichert für die weitere Verwendung
pathToPickle_all_battery_state       = os.path.join(pathToData, 'all_TUDA_battery_states_battery_state.pickle'      )
pathToPickle_all_clean_battery_state = os.path.join(pathToData, 'all_TUDA_battery_states_battery_state_clean.pickle')
batteryDf_preproc.to_pickle(pathToPickle_all_battery_state) #speicher das veränderte DataFrame im pickle format ab, somit kann von diesem weiter gearbeitet werden
batteryDf_clean_preproc.to_pickle(pathToPickle_all_clean_battery_state)

In [ ]:
len(np.where(batteryDf_clean_preproc.battery_state==0)[0])

### Erstelen einer seperaten Datei, um den Zusammenhang zwischen Battery2 und 1 zu analysieren

Entferen aller Daten,
+ die nicht von Provider 35 stammen 
+ die keinen battery2_state enthalten

In [ ]:
battery2AnaDf = batteryDf_clean_preproc.drop(batteryDf_clean_preproc.loc[batteryDf_clean_preproc['provider']!=35].index)
battery2AnaDf.dropna(axis=0, inplace = True, subset = ['battery2_state'])

+ abspeichern des bereinigten DF in pickle-Datei

In [ ]:
# wird als gepickelte .csv Datei gespeichert für die weitere Verwendung
pathToPickle_battery2_state = os.path.join(pathToData, 'TUDA_battery_states_battery2_state.pickle'      )
battery2AnaDf.to_pickle(pathToPickle_battery2_state) #speicher das veränderte DataFrame im pickle format ab, somit kann von diesem weiter gearbeitet werden

___

___

# weiterer Part nur als Speicher!

## weitere Dateien erstellen mit Battery und Battery2 getrennt

+ Aufteilen des Dataframes in Dataframe für battery_state (-> batteryDfbattery_state) und Dataframe für battery2_state (-> batteryDfbattery2_state)
    entfernen von Zeilen mit Batteriewerten < 0 und NaN

In [ ]:
batteryDfbattery_state = batteryDf.drop(['battery2_state'], axis=1)
batteryDfbattery2_state = batteryDf.drop(['battery_state'], axis=1)

In [ ]:
# Batteriezustände = 0 NICHT vernachlässigen, da diese nicht tatsächlich 0 sind --> laut Sprechstunde mit DBCargo --> folglich nur Werte kleiner 0 vernachlässigen
batteryDfbattery_state.drop(batteryDfbattery_state.loc[batteryDfbattery_state['battery_state']<0].index, inplace=True)
batteryDfbattery_state.dropna(inplace=True)

batteryDfbattery2_state.drop(batteryDfbattery2_state.loc[batteryDfbattery2_state['battery2_state']<0].index, inplace=True)
batteryDfbattery2_state.dropna(inplace=True)

In [ ]:
# Liste mit allen wagon_IDs erstellen
wagon_IDList = batteryDf["wagon_ID"]
wagon_IDList = wagon_IDList.drop_duplicates()
wagon_IDList = wagon_IDList.to_numpy()
wagon_IDList = np.sort(wagon_IDList)

In [ ]:
# wagonIDs herausfiltern, die folgende Bedingungen erfüllen: Zeitabstand zwischen einzelnen Messpunkten maximal 20 Tage; mindestens 20 Messpunkte pro Wagon
wagonIDsbattery_state = []

for i, wagon_ID in enumerate(wagon_IDList):
    wagondf = batteryDfbattery_state.loc[batteryDfbattery_state['wagon_ID'] == wagon_ID]
    wagondf = wagondf.sort_values("timestamp_measure_battery")
    timeDiffs = np.diff(wagondf.timestamp_measure_battery.to_numpy())
    timeDiffs = np.floor(timeDiffs/(24*60*60))
    timeDiffs[::-1].sort()
    
    if (len(timeDiffs) < 20):
        continue
        
    for timeDiff in timeDiffs:
        if timeDiff < 20:
            wagonIDsbattery_state.append(wagon_ID)
            break
        else:
            break


In [ ]:
batteryDfbattery_state

In [ ]:
batteryDfbattery_state = batteryDfbattery_state[batteryDfbattery_state['wagon_ID'].isin(wagonIDsbattery_state)]
batteryDfbattery_state

In [ ]:
# wird als gepickelte .csv Datei gespeichert für die weitere Verwendung
pathToPickle_battery_state = os.path.join(pathToData, 'all_TUDA_battery_states_battery_state.pickle')
pathToPickle_battery2_state = os.path.join(pathToData, 'all_TUDA_battery_states_battery2_state.pickle')
batteryDfbattery_state.to_pickle(pathToPickle_battery_state) #speicher das veränderte DataFrame im pickle format ab, somit kann von diesem weiter gearbeitet werden
batteryDfbattery2_state.to_pickle(pathToPickle_battery2_state)
